# 04 · Mamba Denoising QC — Noise Down, Features Intact

This notebook **does not** re-run the Act-of-God denoising pipeline. Instead it inspects the 
packaged outputs under `data/raw/mamba_ssm/`, `data/reference/mamba_ssm/`, and 
`ops/output/data/mamba_checks/` to verify that the shipped spectra preserve readiness-gate 
behaviour. Use it to: (1) confirm the denoised spectra match the recorded pass/fail outcomes in 
Table 2, and (2) visualise key quality metrics such as PSNR, SAM, and ΔSNR heatmaps.

## Prerequisites

- `make reproduce` has been executed so the evaluation artefacts are present locally.
- Heavy dependencies (Torch, `mamba-ssm`) are *optional*. They are only needed if you intend 
  to re-run `python -m mamba_ssm.scripts.evaluate_validation_panel`.
- This notebook only requires the lightweight stack (pandas, numpy, matplotlib, seaborn).

In [ ]:
from __future__ import annotations

import json
from pathlib import Path

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

PROJECT_ROOT = Path(__file__).resolve().parents[1]
PANEL_DIR = PROJECT_ROOT / 'ops' / 'output' / 'data' / 'mamba_checks' / 'panel_eval'
SUMMARY_PATH = PANEL_DIR / 'panel_eval_summary.json'
PER_SPEC_PATH = PANEL_DIR / 'mamba_tiny_uv_best_per_spectrum_metrics.csv'
PER_GROUP_PATH = PANEL_DIR / 'mamba_tiny_uv_best_per_group_summary.csv'
DENOISED_PATH = PROJECT_ROOT / 'data' / 'reference' / 'mamba_ssm' / 'denoised_full_run.csv'

SUMMARY_PATH, PER_SPEC_PATH, PER_GROUP_PATH, DENOISED_PATH

## 1. Readiness Gates Recap

DipAwareLoss was trained in a Noise2Noise regime (see thesis pp. 22–23). The readiness gates 
reported in Table 2 cover:

1. **Panel fidelity** — PSNR_mean, PSNR_std, SAM_mean thresholds.
2. **ROI integrity** — bowl depth / area / centroid behaviour around the absorption features.
3. **Downstream proxy** — dose monotonicity and separability metrics used for screening.

In [ ]:
with SUMMARY_PATH.open(encoding='utf-8') as fh:
    panel_summary = json.load(fh)

panel_summary

In [ ]:
per_group = pd.read_csv(PER_GROUP_PATH)
per_group[['gate', 'metric', 'value', 'threshold', 'pass']]

The expectation (per Table 2) is that the production AoG checkpoint **passes** PSNR_mean and 
SAM_mean, **fails** the PSNR_std tightness gate, flags ROI integrity due to bowl-centroid 
drift, and **passes** the downstream proxy gate. Use the summary above to verify those flags 
without recomputing the denoiser.

In [ ]:
per_spec = pd.read_csv(PER_SPEC_PATH)
per_spec.head()

## 2. ΔSNR Heatmap (Figure 4)

We can approximate Figure 4 by comparing per-spectrum SNR before/after denoising.
The per-spectrum CSV contains both `snr_input` and `snr_output` (dB).

In [ ]:
per_spec['delta_snr_db'] = per_spec['snr_output_db'] - per_spec['snr_input_db']
pivot = per_spec.pivot_table(
    index='dose_id',
    columns='roi',
    values='delta_snr_db',
    aggfunc='median'
)
pivot = pivot.sort_index()
plt.figure(figsize=(8, 4))
sns.heatmap(pivot, annot=True, fmt='.2f', cmap='RdBu_r', center=0)
plt.title('ΔSNR (denoised - raw) by Dose & ROI window')
plt.ylabel('Dose ID')
plt.xlabel('ROI window')
plt.show()

## 3. Optional Deeper Inspection

- Use `DENOISED_PATH` to join denoised spectra with concentration crosswalks for case studies.
- Compare ROI centroid/depth distributions using `per_spec[['roi_centroid_nm', 'roi_depth']]`.
- Re-run the full evaluation with
  `python -m mamba_ssm.scripts.evaluate_validation_panel --manifest data/reference/mamba_ssm/validation_panel.csv`
  after installing Torch + `mamba-ssm[causal-conv1d]`.